# Chapter 22: WSGI and HTTP Fundamentals

**Web Development Fundamentals**

The Hypertext Transfer Protocol (HTTP) is the foundation of data communication on the
World Wide Web. Python provides built-in modules for working with HTTP at every level,
from low-level status codes (`http.HTTPStatus`) to development servers (`http.server`)
to the WSGI specification that underpins frameworks like Flask and Django. Understanding
these primitives is essential before reaching for higher-level abstractions.

## HTTP Protocol Basics: Request/Response Cycle

HTTP is a stateless, text-based protocol built on TCP. Every interaction follows
the same pattern:

1. **Client** sends a **request**: method, path, headers, optional body
2. **Server** processes the request
3. **Server** sends a **response**: status code, headers, body

The request and response are composed of a start line, headers (key-value pairs),
a blank line separator, and an optional body.

In [ ]:
# Anatomy of an HTTP request and response as raw text

http_request: str = (
    "GET /api/users?page=2 HTTP/1.1\r\n"
    "Host: example.com\r\n"
    "Accept: application/json\r\n"
    "User-Agent: PythonClient/1.0\r\n"
    "\r\n"  # Blank line separates headers from body
)

http_response: str = (
    "HTTP/1.1 200 OK\r\n"
    "Content-Type: application/json\r\n"
    "Content-Length: 27\r\n"
    "\r\n"
    '{"users": [], "page": 2}\r\n'
)

print("=== HTTP Request ===")
print(http_request)

print("=== HTTP Response ===")
print(http_response)

# Parse the request line
request_line = http_request.split("\r\n")[0]
method, path, version = request_line.split(" ")
print(f"Method: {method}")
print(f"Path:   {path}")
print(f"Version: {version}")

## HTTP Methods and Their Semantics

HTTP defines several request methods, each with specific semantics:

| Method | Purpose | Safe | Idempotent | Body |
|--------|---------|------|------------|------|
| GET | Retrieve a resource | Yes | Yes | No |
| POST | Create a resource / submit data | No | No | Yes |
| PUT | Replace a resource entirely | No | Yes | Yes |
| PATCH | Partially update a resource | No | No | Yes |
| DELETE | Remove a resource | No | Yes | Optional |
| HEAD | GET without the body | Yes | Yes | No |
| OPTIONS | Describe communication options | Yes | Yes | Optional |

**Safe** means the method does not modify server state. **Idempotent** means
calling it multiple times produces the same result as calling it once.

In [ ]:
from dataclasses import dataclass


@dataclass(frozen=True)
class HTTPMethod:
    """Describes an HTTP method and its properties."""
    name: str
    safe: bool
    idempotent: bool
    has_body: bool
    description: str


methods: list[HTTPMethod] = [
    HTTPMethod("GET", safe=True, idempotent=True, has_body=False,
               description="Retrieve a resource"),
    HTTPMethod("POST", safe=False, idempotent=False, has_body=True,
               description="Create a resource or submit data"),
    HTTPMethod("PUT", safe=False, idempotent=True, has_body=True,
               description="Replace a resource entirely"),
    HTTPMethod("PATCH", safe=False, idempotent=False, has_body=True,
               description="Partially update a resource"),
    HTTPMethod("DELETE", safe=False, idempotent=True, has_body=False,
               description="Remove a resource"),
    HTTPMethod("HEAD", safe=True, idempotent=True, has_body=False,
               description="GET without the response body"),
    HTTPMethod("OPTIONS", safe=True, idempotent=True, has_body=False,
               description="Describe communication options"),
]

for m in methods:
    flags = []
    if m.safe:
        flags.append("safe")
    if m.idempotent:
        flags.append("idempotent")
    if m.has_body:
        flags.append("has body")
    print(f"{m.name:8s} -- {m.description:40s} [{', '.join(flags)}]")

## HTTP Status Codes with `http.HTTPStatus`

Python's `http` module provides the `HTTPStatus` enum with all standard status
codes. Status codes are grouped into five categories:

- **1xx Informational**: Request received, processing continues
- **2xx Success**: Request successfully received and processed
- **3xx Redirection**: Further action needed to complete the request
- **4xx Client Error**: The request contains bad syntax or cannot be fulfilled
- **5xx Server Error**: The server failed to fulfill a valid request

In [ ]:
from http import HTTPStatus
from collections import defaultdict


# Explore commonly used status codes
common_codes: list[HTTPStatus] = [
    HTTPStatus.OK,
    HTTPStatus.CREATED,
    HTTPStatus.NO_CONTENT,
    HTTPStatus.MOVED_PERMANENTLY,
    HTTPStatus.NOT_MODIFIED,
    HTTPStatus.BAD_REQUEST,
    HTTPStatus.UNAUTHORIZED,
    HTTPStatus.FORBIDDEN,
    HTTPStatus.NOT_FOUND,
    HTTPStatus.METHOD_NOT_ALLOWED,
    HTTPStatus.CONFLICT,
    HTTPStatus.UNPROCESSABLE_ENTITY,
    HTTPStatus.INTERNAL_SERVER_ERROR,
    HTTPStatus.BAD_GATEWAY,
    HTTPStatus.SERVICE_UNAVAILABLE,
]

for status in common_codes:
    print(f"{status.value:3d} {status.phrase:30s}  {status.description}")

# Group all status codes by category
print("\n=== Status Code Categories ===")
categories: dict[str, list[HTTPStatus]] = defaultdict(list)
category_names: dict[int, str] = {
    1: "Informational",
    2: "Success",
    3: "Redirection",
    4: "Client Error",
    5: "Server Error",
}

for status in HTTPStatus:
    group = status.value // 100
    categories[category_names[group]].append(status)

for name, statuses in categories.items():
    print(f"  {name}: {len(statuses)} status codes")

## HTTP Headers: Common Request and Response Headers

Headers carry metadata about the request or response. They are case-insensitive
key-value pairs separated by a colon. Some important headers include:

- **Content-Type**: MIME type of the body (e.g., `application/json`)
- **Content-Length**: Size of the body in bytes
- **Accept**: What content types the client can handle
- **Authorization**: Credentials for authentication
- **Cache-Control**: Caching directives

In [ ]:
from dataclasses import dataclass, field


@dataclass
class HTTPHeaders:
    """A simple case-insensitive header collection."""
    _headers: dict[str, str] = field(default_factory=dict)

    def set(self, name: str, value: str) -> None:
        """Set a header (case-insensitive key)."""
        self._headers[name.lower()] = value

    def get(self, name: str, default: str = "") -> str:
        """Get a header value (case-insensitive key)."""
        return self._headers.get(name.lower(), default)

    def items(self) -> list[tuple[str, str]]:
        return list(self._headers.items())

    def __repr__(self) -> str:
        lines = [f"  {k}: {v}" for k, v in self._headers.items()]
        return "Headers(\n" + "\n".join(lines) + "\n)"


# Build typical request headers
request_headers = HTTPHeaders()
request_headers.set("Host", "api.example.com")
request_headers.set("Accept", "application/json")
request_headers.set("Authorization", "Bearer eyJhbGci...")
request_headers.set("User-Agent", "PythonClient/1.0")
request_headers.set("Accept-Encoding", "gzip, deflate")

print("=== Request Headers ===")
print(request_headers)

# Build typical response headers
response_headers = HTTPHeaders()
response_headers.set("Content-Type", "application/json; charset=utf-8")
response_headers.set("Content-Length", "1024")
response_headers.set("Cache-Control", "no-cache, no-store")
response_headers.set("X-Request-Id", "abc-123-def")

print("=== Response Headers ===")
print(response_headers)

# Case-insensitive access
print(f"Content-Type (lowercase lookup): {response_headers.get('content-type')}")
print(f"Content-Type (mixed case):       {response_headers.get('Content-Type')}")

## The WSGI Specification

WSGI (Web Server Gateway Interface, PEP 3333) is the standard interface between
Python web servers and web applications. It defines a simple contract:

- The **application** is a callable that takes two arguments:
  1. `environ` -- a dictionary with CGI-style environment variables
  2. `start_response` -- a callback to begin the HTTP response
- The application returns an iterable of byte strings (the response body).

This simple interface is what allows any WSGI server (Gunicorn, uWSGI, mod_wsgi)
to run any WSGI application (Flask, Django, Pyramid).

In [ ]:
from typing import Callable, Iterable


# Type aliases for WSGI
Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]
WSGIApp = Callable[[Environ, StartResponse], Iterable[bytes]]


def hello_app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    """The simplest possible WSGI application."""
    status: str = "200 OK"
    headers: list[tuple[str, str]] = [
        ("Content-Type", "text/plain; charset=utf-8"),
    ]
    start_response(status, headers)
    return [b"Hello, WSGI World!"]


# Simulate calling the WSGI app (what a server does internally)
captured_status: str = ""
captured_headers: list[tuple[str, str]] = []


def mock_start_response(
    status: str,
    response_headers: list[tuple[str, str]],
) -> Callable[..., object]:
    """Mock start_response that captures the status and headers."""
    global captured_status, captured_headers
    captured_status = status
    captured_headers = response_headers
    return lambda *args: None  # write() callable (rarely used)


# Build a minimal environ dict
environ: Environ = {
    "REQUEST_METHOD": "GET",
    "PATH_INFO": "/",
    "SERVER_NAME": "localhost",
    "SERVER_PORT": "8000",
    "HTTP_HOST": "localhost:8000",
}

# Call the application
body_parts: Iterable[bytes] = hello_app(environ, mock_start_response)
body: bytes = b"".join(body_parts)

print(f"Status:  {captured_status}")
print(f"Headers: {captured_headers}")
print(f"Body:    {body.decode()}")

## The WSGI `environ` Dictionary

The `environ` dict contains CGI-style variables plus WSGI-specific keys.
Understanding its contents is crucial for parsing requests:

- **REQUEST_METHOD**: GET, POST, PUT, DELETE, etc.
- **PATH_INFO**: The URL path (e.g., `/api/users`)
- **QUERY_STRING**: Everything after `?` in the URL
- **CONTENT_TYPE**: MIME type of the request body
- **CONTENT_LENGTH**: Size of the request body
- **HTTP_***: Request headers (e.g., `HTTP_HOST`, `HTTP_ACCEPT`)
- **wsgi.input**: A file-like object for reading the request body
- **wsgi.errors**: A file-like object for error output

In [ ]:
import io
from urllib.parse import parse_qs


def build_environ(
    method: str = "GET",
    path: str = "/",
    query_string: str = "",
    body: bytes = b"",
    content_type: str = "",
    headers: dict[str, str] | None = None,
) -> Environ:
    """Build a realistic WSGI environ dict for testing."""
    environ: Environ = {
        # CGI variables
        "REQUEST_METHOD": method,
        "SCRIPT_NAME": "",
        "PATH_INFO": path,
        "QUERY_STRING": query_string,
        "SERVER_NAME": "localhost",
        "SERVER_PORT": "8000",
        "SERVER_PROTOCOL": "HTTP/1.1",
        "CONTENT_TYPE": content_type,
        "CONTENT_LENGTH": str(len(body)),
        # WSGI variables
        "wsgi.version": (1, 0),
        "wsgi.url_scheme": "http",
        "wsgi.input": io.BytesIO(body),
        "wsgi.errors": io.StringIO(),
        "wsgi.multithread": False,
        "wsgi.multiprocess": False,
        "wsgi.run_once": False,
    }
    # Add HTTP_ headers
    if headers:
        for key, value in headers.items():
            environ_key = "HTTP_" + key.upper().replace("-", "_")
            environ[environ_key] = value
    return environ


# Build environ for a GET request with query parameters
env = build_environ(
    method="GET",
    path="/api/users",
    query_string="page=2&limit=10&sort=name",
    headers={
        "Host": "api.example.com",
        "Accept": "application/json",
        "Authorization": "Bearer token123",
    },
)

print("=== WSGI Environ (CGI Variables) ===")
cgi_keys = [
    "REQUEST_METHOD", "PATH_INFO", "QUERY_STRING",
    "SERVER_NAME", "SERVER_PORT", "SERVER_PROTOCOL",
]
for key in cgi_keys:
    print(f"  {key}: {env[key]}")

print("\n=== HTTP Headers (from environ) ===")
for key, value in env.items():
    if isinstance(key, str) and key.startswith("HTTP_"):
        header_name = key[5:].replace("_", "-").title()
        print(f"  {header_name}: {value}")

# Parse query string
print("\n=== Parsed Query Parameters ===")
qs = str(env["QUERY_STRING"])
params: dict[str, list[str]] = parse_qs(qs)
for key, values in params.items():
    print(f"  {key}: {values}")

## Building a WSGI App That Parses Requests

A practical WSGI application inspects the method, path, query string, and
body to decide what to do. Below we build an application that routes based
on `PATH_INFO` and handles different HTTP methods.

In [ ]:
import json
import io
from http import HTTPStatus
from typing import Callable, Iterable
from urllib.parse import parse_qs


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]


def read_body(environ: Environ) -> bytes:
    """Read the request body from wsgi.input."""
    content_length_str = str(environ.get("CONTENT_LENGTH", "0") or "0")
    try:
        content_length = int(content_length_str)
    except ValueError:
        content_length = 0
    wsgi_input = environ.get("wsgi.input")
    if wsgi_input and content_length > 0:
        assert hasattr(wsgi_input, "read")
        return wsgi_input.read(content_length)  # type: ignore[union-attr]
    return b""


def json_response(
    data: object,
    status: HTTPStatus,
    start_response: StartResponse,
) -> Iterable[bytes]:
    """Helper to return a JSON response."""
    body: bytes = json.dumps(data, indent=2).encode("utf-8")
    start_response(
        f"{status.value} {status.phrase}",
        [
            ("Content-Type", "application/json; charset=utf-8"),
            ("Content-Length", str(len(body))),
        ],
    )
    return [body]


def demo_app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    """A WSGI app that inspects and echoes request details."""
    method = str(environ.get("REQUEST_METHOD", "GET"))
    path = str(environ.get("PATH_INFO", "/"))
    query_string = str(environ.get("QUERY_STRING", ""))

    if path == "/echo":
        body = read_body(environ)
        response_data = {
            "method": method,
            "path": path,
            "query_params": parse_qs(query_string),
            "body": body.decode("utf-8", errors="replace"),
            "content_type": str(environ.get("CONTENT_TYPE", "")),
        }
        return json_response(response_data, HTTPStatus.OK, start_response)

    elif path == "/health":
        return json_response({"status": "healthy"}, HTTPStatus.OK, start_response)

    else:
        return json_response(
            {"error": "Not Found", "path": path},
            HTTPStatus.NOT_FOUND,
            start_response,
        )


# Test the app with different requests
def call_app(app: Callable[..., Iterable[bytes]], environ: Environ) -> None:
    """Simulate calling a WSGI app and print the results."""
    result_status = ""
    result_headers: list[tuple[str, str]] = []

    def start_response(
        status: str, headers: list[tuple[str, str]]
    ) -> Callable[..., object]:
        nonlocal result_status, result_headers
        result_status = status
        result_headers = headers
        return lambda *args: None

    body = b"".join(app(environ, start_response))
    print(f"  Status: {result_status}")
    print(f"  Body:   {body.decode()}")


print("=== GET /health ===")
call_app(demo_app, build_environ(path="/health"))

print("\n=== GET /echo?name=alice&role=admin ===")
call_app(demo_app, build_environ(path="/echo", query_string="name=alice&role=admin"))

print("\n=== POST /echo with JSON body ===")
call_app(
    demo_app,
    build_environ(
        method="POST",
        path="/echo",
        body=b'{"message": "hello"}',
        content_type="application/json",
    ),
)

print("\n=== GET /unknown ===")
call_app(demo_app, build_environ(path="/unknown"))

## Response Building: Status Line, Headers, Body

When constructing HTTP responses, you must:

1. Set the correct **status code** (matching the semantic outcome)
2. Include appropriate **headers** (Content-Type, Content-Length, caching, etc.)
3. Encode the **body** as bytes (HTTP transmits raw bytes, not strings)

Below is a `ResponseBuilder` that makes response construction explicit and safe.

In [ ]:
import json
from http import HTTPStatus
from dataclasses import dataclass, field


@dataclass
class Response:
    """Encapsulates an HTTP response for WSGI."""
    status: HTTPStatus = HTTPStatus.OK
    headers: dict[str, str] = field(default_factory=dict)
    body: bytes = b""

    @property
    def status_line(self) -> str:
        return f"{self.status.value} {self.status.phrase}"

    @property
    def header_list(self) -> list[tuple[str, str]]:
        return list(self.headers.items())

    @classmethod
    def text(cls, content: str, status: HTTPStatus = HTTPStatus.OK) -> "Response":
        """Create a plain text response."""
        body = content.encode("utf-8")
        return cls(
            status=status,
            headers={
                "Content-Type": "text/plain; charset=utf-8",
                "Content-Length": str(len(body)),
            },
            body=body,
        )

    @classmethod
    def html(cls, content: str, status: HTTPStatus = HTTPStatus.OK) -> "Response":
        """Create an HTML response."""
        body = content.encode("utf-8")
        return cls(
            status=status,
            headers={
                "Content-Type": "text/html; charset=utf-8",
                "Content-Length": str(len(body)),
            },
            body=body,
        )

    @classmethod
    def json_data(
        cls, data: object, status: HTTPStatus = HTTPStatus.OK
    ) -> "Response":
        """Create a JSON response."""
        body = json.dumps(data, indent=2).encode("utf-8")
        return cls(
            status=status,
            headers={
                "Content-Type": "application/json; charset=utf-8",
                "Content-Length": str(len(body)),
            },
            body=body,
        )

    @classmethod
    def redirect(cls, location: str, permanent: bool = False) -> "Response":
        """Create a redirect response."""
        status = (
            HTTPStatus.MOVED_PERMANENTLY if permanent
            else HTTPStatus.FOUND
        )
        return cls(
            status=status,
            headers={"Location": location, "Content-Length": "0"},
            body=b"",
        )


# Demonstrate different response types
responses: list[tuple[str, Response]] = [
    ("Plain text", Response.text("Hello, World!")),
    ("HTML", Response.html("<h1>Welcome</h1><p>Hello!</p>")),
    ("JSON", Response.json_data({"users": ["alice", "bob"], "count": 2})),
    ("Redirect", Response.redirect("/new-location")),
    ("Not Found", Response.text("Page not found", HTTPStatus.NOT_FOUND)),
]

for label, resp in responses:
    print(f"=== {label} ===")
    print(f"  Status:  {resp.status_line}")
    print(f"  Headers: {resp.header_list}")
    print(f"  Body:    {resp.body[:80]!r}")
    print()

## Development Server with `http.server`

Python's `http.server` module provides a simple HTTP server suitable for
development and testing. It can serve static files out of the box, and can
be extended with custom request handlers. The module also includes
`CGIHTTPRequestHandler` for CGI scripts and can be used as a quick WSGI
host via `wsgiref`.

In [ ]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import threading
import urllib.request
import json


class SimpleAPIHandler(BaseHTTPRequestHandler):
    """A minimal HTTP request handler demonstrating the http.server API."""

    def do_GET(self) -> None:
        """Handle GET requests."""
        if self.path == "/api/info":
            data = {
                "server": "SimpleAPIHandler",
                "method": "GET",
                "path": self.path,
                "client": self.client_address[0],
            }
            body = json.dumps(data, indent=2).encode("utf-8")
            self.send_response(200)
            self.send_header("Content-Type", "application/json")
            self.send_header("Content-Length", str(len(body)))
            self.end_headers()
            self.wfile.write(body)
        else:
            self.send_error(404, "Not Found")

    def log_message(self, format: str, *args: object) -> None:
        """Suppress default logging to keep notebook output clean."""
        pass


# Start a development server in a background thread
server = HTTPServer(("127.0.0.1", 0), SimpleAPIHandler)
port = server.server_address[1]
print(f"Development server running on http://127.0.0.1:{port}")

server_thread = threading.Thread(target=server.serve_forever, daemon=True)
server_thread.start()

# Make a request to the server
try:
    url = f"http://127.0.0.1:{port}/api/info"
    with urllib.request.urlopen(url) as response:
        print(f"\nResponse status: {response.status}")
        print(f"Content-Type: {response.headers['Content-Type']}")
        body = json.loads(response.read())
        print(f"Body: {json.dumps(body, indent=2)}")

    # Request a non-existent path
    try:
        url_404 = f"http://127.0.0.1:{port}/nonexistent"
        urllib.request.urlopen(url_404)
    except urllib.error.HTTPError as e:
        print(f"\n404 test: status={e.code}, reason={e.reason}")

finally:
    server.shutdown()
    print("\nServer shut down.")

## Running a WSGI App with `wsgiref`

The `wsgiref` module in the standard library provides a reference WSGI server
implementation. It is suitable for development and testing. The `make_server()`
function creates a server that hosts any WSGI-compliant application.

In [ ]:
from wsgiref.simple_server import make_server
import threading
import urllib.request
import json
from typing import Callable, Iterable
from http import HTTPStatus


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]


def wsgiref_demo_app(
    environ: Environ, start_response: StartResponse
) -> Iterable[bytes]:
    """A WSGI app served by wsgiref."""
    path = str(environ.get("PATH_INFO", "/"))
    method = str(environ.get("REQUEST_METHOD", "GET"))

    response_data = {
        "message": f"Handled {method} {path} via wsgiref",
        "server": "wsgiref.simple_server",
        "wsgi_version": list(environ.get("wsgi.version", (1, 0))),  # type: ignore[arg-type]
    }
    body = json.dumps(response_data, indent=2).encode("utf-8")

    start_response(
        f"{HTTPStatus.OK.value} {HTTPStatus.OK.phrase}",
        [
            ("Content-Type", "application/json"),
            ("Content-Length", str(len(body))),
        ],
    )
    return [body]


# Create and start the wsgiref server
httpd = make_server("127.0.0.1", 0, wsgiref_demo_app)
port = httpd.server_address[1]
print(f"wsgiref server running on http://127.0.0.1:{port}")

server_thread = threading.Thread(target=httpd.serve_forever, daemon=True)
server_thread.start()

# Test the WSGI app through the actual HTTP server
try:
    for path in ["/", "/api/data", "/users/123"]:
        url = f"http://127.0.0.1:{port}{path}"
        with urllib.request.urlopen(url) as resp:
            data = json.loads(resp.read())
            print(f"GET {path} -> {data['message']}")
finally:
    httpd.shutdown()
    print("\nwsgiref server shut down.")

## Request Parsing: A Complete Example

Pulling it all together, here is a `Request` dataclass that extracts all
relevant information from a WSGI `environ` dict -- method, path, query
parameters, headers, and body -- into a clean, typed interface.

In [ ]:
import io
import json
from dataclasses import dataclass, field
from urllib.parse import parse_qs


@dataclass
class Request:
    """Parsed HTTP request extracted from a WSGI environ."""
    method: str
    path: str
    query_params: dict[str, list[str]]
    headers: dict[str, str]
    body: bytes
    content_type: str

    @classmethod
    def from_environ(cls, environ: dict[str, object]) -> "Request":
        """Parse a WSGI environ dict into a Request object."""
        method = str(environ.get("REQUEST_METHOD", "GET"))
        path = str(environ.get("PATH_INFO", "/"))
        query_string = str(environ.get("QUERY_STRING", ""))
        content_type = str(environ.get("CONTENT_TYPE", ""))

        # Parse content length and read body
        try:
            content_length = int(str(environ.get("CONTENT_LENGTH", "0") or "0"))
        except ValueError:
            content_length = 0

        wsgi_input = environ.get("wsgi.input")
        body = b""
        if wsgi_input and content_length > 0 and hasattr(wsgi_input, "read"):
            body = wsgi_input.read(content_length)  # type: ignore[union-attr]

        # Extract HTTP headers from environ
        headers: dict[str, str] = {}
        for key, value in environ.items():
            if isinstance(key, str) and key.startswith("HTTP_"):
                header_name = key[5:].replace("_", "-").title()
                headers[header_name] = str(value)

        return cls(
            method=method,
            path=path,
            query_params=parse_qs(query_string),
            headers=headers,
            body=body,
            content_type=content_type,
        )

    @property
    def json(self) -> object:
        """Parse the body as JSON."""
        return json.loads(self.body) if self.body else None

    def get_query(self, key: str, default: str = "") -> str:
        """Get a single query parameter value."""
        values = self.query_params.get(key, [])
        return values[0] if values else default


# Test with a simulated POST request
post_body = json.dumps({"username": "alice", "email": "alice@example.com"}).encode()
test_environ: dict[str, object] = {
    "REQUEST_METHOD": "POST",
    "PATH_INFO": "/api/users",
    "QUERY_STRING": "notify=true&format=json",
    "CONTENT_TYPE": "application/json",
    "CONTENT_LENGTH": str(len(post_body)),
    "wsgi.input": io.BytesIO(post_body),
    "HTTP_HOST": "api.example.com",
    "HTTP_ACCEPT": "application/json",
    "HTTP_AUTHORIZATION": "Bearer secret-token",
    "HTTP_X_REQUEST_ID": "req-abc-123",
}

req = Request.from_environ(test_environ)
print(f"Method:       {req.method}")
print(f"Path:         {req.path}")
print(f"Content-Type: {req.content_type}")
print(f"Query params: {req.query_params}")
print(f"  notify:     {req.get_query('notify')}")
print(f"  format:     {req.get_query('format')}")
print(f"  missing:    {req.get_query('missing', 'default-value')}")
print(f"Headers:      {req.headers}")
print(f"Body (raw):   {req.body}")
print(f"Body (JSON):  {req.json}")

## Summary

This notebook covered the foundations of HTTP and WSGI in Python:

1. **HTTP request/response cycle**: The text-based protocol with method, path,
   headers, status codes, and body
2. **HTTP methods**: GET, POST, PUT, PATCH, DELETE and their semantic properties
   (safe, idempotent, body)
3. **Status codes**: The `http.HTTPStatus` enum organized into 1xx-5xx categories
4. **Headers**: Case-insensitive key-value metadata for requests and responses
5. **WSGI specification**: The `environ` dict, `start_response` callable, and
   the application callable interface (PEP 3333)
6. **Request parsing**: Extracting method, path, query string, headers, and body
   from the WSGI environ
7. **Response building**: Constructing status lines, headers, and encoded bodies
8. **Development servers**: `http.server` for simple handlers and `wsgiref` for
   hosting WSGI applications

The next notebook covers URL routing, template rendering, and form handling --
building on these WSGI primitives to create more structured web applications.